In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from keras.preprocessing import sequence
from sklearn.preprocessing import label_binarize
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution1D,MaxPooling1D, Flatten, Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU ,Bidirectional, CuDNNLSTM ,BatchNormalization ,Flatten, Attention
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score, matthews_corrcoef, roc_curve, auc, roc_auc_score)
from sklearn import metrics
from sklearn.preprocessing import Normalizer , LabelEncoder , StandardScaler
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import math
import time
from itertools import cycle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv(r'/content/drive/MyDrive/MSCAD/MSCAD.csv', header=0)

In [ ]:
c=[ 3, 7, 12, 13, 15, 16, 22, 23, 24, 25, 33, 34, 36, 39, 50, 52, 56, 57, 66]
data=data[data.columns[c]]

In [ ]:
data

,'TotLen Fwd Pkts','Fwd Pkt Len Mean','Bwd Pkt Len Std','Flow Byts/s','Flow IAT Mean','Flow IAT Std','Fwd IAT Max','Fwd IAT Min','Bwd IAT Tot','Bwd IAT Mean','Fwd Pkts/s','Bwd Pkts/s','Pkt Len Max','Pkt Len Var','Fwd Seg Size Avg','Subflow Fwd Pkts','Init Bwd Win Byts','Fwd Act Data Pkts',Label
0,110,55.0000,168.5995,3.208169e+05,2.530000e+02,1.595306e+02,487,487,1518,379.5,1317.5231,3293.8076,377,17797.5536,55.0000,2,46,1,Brute_Force
1,168,42.0000,744.4771,7.916525e+05,5.358182e+02,7.749130e+02,2675,677,5894,842.0,678.6563,1357.3125,1460,396851.0769,42.0000,4,54,1,Brute_Force
2,0,0.0000,0.0000,0.000000e+00,2.720000e+02,0.000000e+00,0,0,0,0.0,3676.4706,3676.4706,0,0.0000,0.0000,1,4106,0,Brute_Force
3,322,80.5000,750.2544,1.821524e+06,2.373636e+02,2.376124e+02,1368,367,2611,373.0,1531.9801,3063.9602,1460,396786.3077,80.5000,4,54,1,Brute_Force
4,0,0.0000,0.0000,0.000000e+00,2.940000e+02,0.000000e+00,0,0,0,0.0,3401.3605,3401.3605,0,0.0000,0.0000,1,4106,0,Brute_Force
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128794,517,172.3333,0.0000,7.499057e+03,1.723550e+04,2.928714e+04,7697,304,60845,60845.0,43.5148,29.0099,517,44548.1667,172.3333,3,65535,1,Normal
128795,1277,638.5000,958.8368,4.157784e+04,2.110900e+04,3.651164e+04,63327,63327,58,58.0,31.5821,31.5821,1356,531444.2000,638.5000,2,0,1,Normal
128796,517,172.3333,0.0000,3.717793e+03,2.781220e+04,3.653110e+04,71498,254,135929,67964.5,21.5733,21.5733,517,38184.1429,172.3333,3,261,1,Normal
128797,0,0.0000,0.4472,4.620000e-02,3.606975e+06,4.534670e+06,10100000,10100000,21600000,5410462.0,0.0924,0.2310,1,0.1250,0.0000,2,257,0,Normal


In [ ]:
data[data.columns[18]].value_counts()

Brute_Force    88502
Normal         28502
Port_Scan      11081
HTTP_DDoS        641
ICMP_Flood        45
Web_Crwling       28
Name: Label, dtype: int64

In [ ]:
X = data.iloc[:,0:18]
Y = data['Label']

In [ ]:
X

,'TotLen Fwd Pkts','Fwd Pkt Len Mean','Bwd Pkt Len Std','Flow Byts/s','Flow IAT Mean','Flow IAT Std','Fwd IAT Max','Fwd IAT Min','Bwd IAT Tot','Bwd IAT Mean','Fwd Pkts/s','Bwd Pkts/s','Pkt Len Max','Pkt Len Var','Fwd Seg Size Avg','Subflow Fwd Pkts','Init Bwd Win Byts','Fwd Act Data Pkts'
0,110,55.0000,168.5995,3.208169e+05,2.530000e+02,1.595306e+02,487,487,1518,379.5,1317.5231,3293.8076,377,17797.5536,55.0000,2,46,1
1,168,42.0000,744.4771,7.916525e+05,5.358182e+02,7.749130e+02,2675,677,5894,842.0,678.6563,1357.3125,1460,396851.0769,42.0000,4,54,1
2,0,0.0000,0.0000,0.000000e+00,2.720000e+02,0.000000e+00,0,0,0,0.0,3676.4706,3676.4706,0,0.0000,0.0000,1,4106,0
3,322,80.5000,750.2544,1.821524e+06,2.373636e+02,2.376124e+02,1368,367,2611,373.0,1531.9801,3063.9602,1460,396786.3077,80.5000,4,54,1
4,0,0.0000,0.0000,0.000000e+00,2.940000e+02,0.000000e+00,0,0,0,0.0,3401.3605,3401.3605,0,0.0000,0.0000,1,4106,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128794,517,172.3333,0.0000,7.499057e+03,1.723550e+04,2.928714e+04,7697,304,60845,60845.0,43.5148,29.0099,517,44548.1667,172.3333,3,65535,1
128795,1277,638.5000,958.8368,4.157784e+04,2.110900e+04,3.651164e+04,63327,63327,58,58.0,31.5821,31.5821,1356,531444.2000,638.5000,2,0,1
128796,517,172.3333,0.0000,3.717793e+03,2.781220e+04,3.653110e+04,71498,254,135929,67964.5,21.5733,21.5733,517,38184.1429,172.3333,3,261,1
128797,0,0.0000,0.4472,4.620000e-02,3.606975e+06,4.534670e+06,10100000,10100000,21600000,5410462.0,0.0924,0.2310,1,0.1250,0.0000,2,257,0


In [ ]:
Y

0         Brute_Force
1         Brute_Force
2         Brute_Force
3         Brute_Force
4         Brute_Force
             ...     
128794         Normal
128795         Normal
128796         Normal
128797         Normal
128798         Normal
Name: Label, Length: 128799, dtype: object

In [ ]:
scaler = StandardScaler().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)

In [ ]:
label=LabelEncoder()
Y=label.fit_transform(Y)


In [ ]:
Y

array([0, 0, 0, ..., 3, 3, 3])

In [ ]:
df=pd.DataFrame(trainX)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-0.013860,-0.002638,-0.297168,0.263236,-0.201793,-0.223605,-0.262023,-0.170951,-0.191990,-0.144498,0.421145,0.523996,-0.264379,-0.344355,-0.002638,-0.019815,-0.340588,-0.016919
1,-0.013832,-0.120727,1.313647,1.129595,-0.201762,-0.223471,-0.261847,-0.170926,-0.191751,-0.144392,0.133563,0.027806,1.245935,0.664355,-0.120727,-0.018500,-0.338914,-0.016919
2,-0.013914,-0.502245,-0.768766,-0.327081,-0.201791,-0.223640,-0.262063,-0.171015,-0.192073,-0.144585,1.483008,0.622046,-0.790131,-0.391716,-0.502245,-0.020473,0.509332,-0.017579
3,-0.013757,0.228998,1.329807,3.024606,-0.201795,-0.223588,-0.261952,-0.170967,-0.191930,-0.144500,0.517681,0.465102,1.245935,0.664182,0.228998,-0.018500,-0.338914,-0.016919
4,-0.013914,-0.502245,-0.768766,-0.327081,-0.201788,-0.223640,-0.262063,-0.171015,-0.192073,-0.144585,1.359169,0.551555,-0.790131,-0.391716,-0.502245,-0.020473,0.509332,-0.017579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128794,-0.013662,1.063190,-0.768766,-0.313283,-0.199960,-0.217234,-0.261441,-0.170975,-0.188745,-0.130657,-0.152341,-0.312547,-0.069140,-0.273168,1.063190,-0.019158,13.368871,-0.016919
128795,-0.013292,5.297738,1.913243,-0.250576,-0.199541,-0.215654,-0.256951,-0.162718,-0.192070,-0.144572,-0.157713,-0.311888,1.100900,1.022524,5.297738,-0.019815,-0.350218,-0.016919
128796,-0.013662,1.063190,-0.768766,-0.320240,-0.198818,-0.215650,-0.256291,-0.170982,-0.184639,-0.129028,-0.162218,-0.314453,-0.069140,-0.290103,1.063190,-0.019158,-0.295580,-0.016919
128797,-0.013914,-0.502245,-0.767515,-0.327081,0.187550,0.768236,0.553258,1.152318,0.989177,1.093897,-0.171887,-0.319921,-0.788736,-0.391716,-0.502245,-0.019815,-0.296418,-0.017579


In [ ]:
df[18]=Y

In [ ]:
df[18].value_counts()

0    88502
3    28502
4    11081
1      641
2       45
5       28
Name: 18, dtype: int64

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-0.013860,-0.002638,-0.297168,0.263236,-0.201793,-0.223605,-0.262023,-0.170951,-0.191990,-0.144498,0.421145,0.523996,-0.264379,-0.344355,-0.002638,-0.019815,-0.340588,-0.016919,0
1,-0.013832,-0.120727,1.313647,1.129595,-0.201762,-0.223471,-0.261847,-0.170926,-0.191751,-0.144392,0.133563,0.027806,1.245935,0.664355,-0.120727,-0.018500,-0.338914,-0.016919,0
2,-0.013914,-0.502245,-0.768766,-0.327081,-0.201791,-0.223640,-0.262063,-0.171015,-0.192073,-0.144585,1.483008,0.622046,-0.790131,-0.391716,-0.502245,-0.020473,0.509332,-0.017579,0
3,-0.013757,0.228998,1.329807,3.024606,-0.201795,-0.223588,-0.261952,-0.170967,-0.191930,-0.144500,0.517681,0.465102,1.245935,0.664182,0.228998,-0.018500,-0.338914,-0.016919,0
4,-0.013914,-0.502245,-0.768766,-0.327081,-0.201788,-0.223640,-0.262063,-0.171015,-0.192073,-0.144585,1.359169,0.551555,-0.790131,-0.391716,-0.502245,-0.020473,0.509332,-0.017579,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128794,-0.013662,1.063190,-0.768766,-0.313283,-0.199960,-0.217234,-0.261441,-0.170975,-0.188745,-0.130657,-0.152341,-0.312547,-0.069140,-0.273168,1.063190,-0.019158,13.368871,-0.016919,3
128795,-0.013292,5.297738,1.913243,-0.250576,-0.199541,-0.215654,-0.256951,-0.162718,-0.192070,-0.144572,-0.157713,-0.311888,1.100900,1.022524,5.297738,-0.019815,-0.350218,-0.016919,3
128796,-0.013662,1.063190,-0.768766,-0.320240,-0.198818,-0.215650,-0.256291,-0.170982,-0.184639,-0.129028,-0.162218,-0.314453,-0.069140,-0.290103,1.063190,-0.019158,-0.295580,-0.016919,3
128797,-0.013914,-0.502245,-0.767515,-0.327081,0.187550,0.768236,0.553258,1.152318,0.989177,1.093897,-0.171887,-0.319921,-0.788736,-0.391716,-0.502245,-0.019815,-0.296418,-0.017579,3


In [ ]:
y_train=traindata[66].values
y_test=testdata[66].values

In [ ]:
traindata, testdata = train_test_split(df,test_size=0.20,random_state=42)

In [ ]:
traindata[18].value_counts()

0    70642
3    22900
4     8916
1      523
2       35
5       23
Name: 18, dtype: int64

In [ ]:
testdata[18].value_counts()

0    17860
3     5602
4     2165
1      118
2       10
5        5
Name: 18, dtype: int64

In [ ]:
df.to_csv(r'mscad_std.csv')

In [ ]:
trainX=traindata.iloc[:,0:18].values
testX=testdata.iloc[:,0:18].values

y_train=to_categorical(traindata[18])
y_test=to_categorical(testdata[18])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
dt=DecisionTreeClassifier()

In [ ]:
#check each classifier accuracy here
clf=rf
clf.fit(trainX,y_train)
y_pred=clf.predict(testX)
acc=accuracy_score(y_test,y_pred)
prc=precision_score(y_test,y_pred, average="weighted")
rc=recall_score(y_test,y_pred, average="weighted")
f1=f1_score(y_test,y_pred, average="weighted")
print(rf)
print('Accuracy: ',acc, '\nPrecision: ' ,prc, '\nRecall: ', rc,'\nF1-Score: ', f1)
print(acc)


In [ ]:
print(metrics.classification_report(y_test,y_pred))
cm=metrics.confusion_matrix(np.argmax(y_test,axis=1), np.argmax(y_pred,axis=1))
print(cm)

In [ ]:
from sklearn.metrics import roc_auc_score
ROC_AUC_SCORE=roc_auc_score(y_test, y_pred,multi_class='ovr')
print('ROC_AUC_SCORE : ',ROC_AUC_SCORE)

In [ ]:
import keras.backend as K
from keras.layers import Layer
from keras.layers import Input
from keras import initializers, regularizers, constraints

In [ ]:
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [ ]:
y_train

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], trainX.shape[1],1))
X_test=np.reshape(testX, (testX.shape[0], testX.shape[1],1))

print(X_train.shape)


batch_size = 32

(103039, 18, 1)


In [ ]:
model = Sequential()
model.add(LSTM(64,return_sequences=True, input_shape=(18,1)))
model.add(attention())
model.add(Dense(32,activation='relu'))
model.add(Dense(6, activation="softmax"))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 18, 64)            16896     
                                                                 
 attention (attention)       (None, 64)                82        
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 19,256
Trainable params: 19,256
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time
checkpointer = callbacks.ModelCheckpoint(filepath="model18.hdf5", verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')
csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
m=model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test,y_test), callbacks=[checkpointer,csv_logger])

Epoch 1/100
1609/1610 [============================>.] - ETA: 0s - loss: 0.0079 - accuracy: 0.9979
Epoch 1: val_accuracy improved from -inf to 0.99720, saving model to kddresults/nslkddmc/checkpoint-01.hdf5
1610/1610 [==============================] - 25s 16ms/step - loss: 0.0079 - accuracy: 0.9979 - val_loss: 0.0192 - val_accuracy: 0.9972
Epoch 2/100
1607/1610 [============================>.] - ETA: 0s - loss: 0.0073 - accuracy: 0.9982
Epoch 2: val_accuracy did not improve from 0.99720
1610/1610 [==============================] - 28s 17ms/step - loss: 0.0073 - accuracy: 0.9982 - val_loss: 0.0189 - val_accuracy: 0.9971
Epoch 3/100
1610/1610 [==============================] - ETA: 0s - loss: 0.0077 - accuracy: 0.9981
Epoch 3: val_accuracy did not improve from 0.99720
1610/1610 [==============================] - 25s 16ms/step - loss: 0.0077 - accuracy: 0.9981 - val_loss: 0.0187 - val_accuracy: 0.9970
Epoch 4/100
1609/1610 [============================>.] - ETA: 0s - loss: 0.0071 - accura

In [ ]:
%%time
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/nslkddmc2/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')
csv_logger = CSVLogger('training_set_iranalysis2.csv',separator=',', append=False)
m1=model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test,y_test), callbacks=[checkpointer,csv_logger])

Epoch 1/50
1608/1610 [============================>.] - ETA: 0s - loss: 0.0076 - accuracy: 0.9981
Epoch 1: val_accuracy improved from -inf to 0.99709, saving model to kddresults/nslkddmc2/checkpoint-01.hdf5
1610/1610 [==============================] - 26s 16ms/step - loss: 0.0076 - accuracy: 0.9980 - val_loss: 0.0214 - val_accuracy: 0.9971
Epoch 2/50
1608/1610 [============================>.] - ETA: 0s - loss: 0.0072 - accuracy: 0.9982
Epoch 2: val_accuracy did not improve from 0.99709
1610/1610 [==============================] - 26s 16ms/step - loss: 0.0072 - accuracy: 0.9982 - val_loss: 0.0215 - val_accuracy: 0.9969
Epoch 3/50
1608/1610 [============================>.] - ETA: 0s - loss: 0.0073 - accuracy: 0.9981
Epoch 3: val_accuracy did not improve from 0.99709
1610/1610 [==============================] - 26s 16ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 0.0223 - val_accuracy: 0.9969
Epoch 4/50
1608/1610 [============================>.] - ETA: 0s - loss: 0.0071 - accuracy:

In [ ]:
model.load_weights('/content/kddresults/nslkddmc/checkpoint-47.hdf5')

In [ ]:

loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.3f, Accuracy: %.3f%%" % (loss, accuracy*100))
#y_pred = model.predict_classes(X_test)


805/805 [==============================] - 4s 5ms/step - loss: 0.0203 - accuracy: 0.9973

Loss: 0.020, Accuracy: 99.732%


In [ ]:
%%time
y_pred=model.predict(X_test) 
classes_x=np.argmax(y_pred,axis=1)
class_x=np.argmax(y_pred.round(),axis=1)

CPU times: user 3.78 s, sys: 215 ms, total: 3.99 s
Wall time: 3.34 s


In [ ]:
y_true=np.argmax(y_test,axis=1)

In [ ]:
recall = recall_score(y_true, classes_x , average="weighted")
precision = precision_score(y_true, classes_x , average="weighted")
f1= f1_score(y_true, classes_x , average="weighted")

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true,classes_x))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17860
           1       0.91      0.96      0.93       118
           2       1.00      0.70      0.82        10
           3       1.00      1.00      1.00      5602
           4       0.99      0.99      0.99      2165
           5       0.00      0.00      0.00         5

    accuracy                           1.00     25760
   macro avg       0.82      0.77      0.79     25760
weighted avg       1.00      1.00      1.00     25760



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.6f" %accuracy)
print("recall")
print("%.6f" %recall)
print("precision")
print("%.6f" %precision)
print("f1score")
print("%.6f" %f1)
cm = metrics.confusion_matrix(y_true, classes_x)
print("==============================================")
print (cm)

confusion matrix
----------------------------------------------
accuracy
0.997321
recall
0.997321
precision
0.997144
f1score
0.997219
[[17854     0     0     4     2     0]
 [    1   113     0     3     1     0]
 [    0     0     7     2     1     0]
 [    3     0     0  5584    15     0]
 [   16    11     0     5  2133     0]
 [    0     0     0     2     3     0]]


In [ ]:
testdata[testdata.columns[18]].value_counts()

0    17860
3     5602
4     2165
1      118
2       10
5        5
Name: 18, dtype: int64

In [ ]:

print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.6f" %accuracy)
print("recall")
print("%.6f" %recall)
print("precision")
print("%.6f" %precision)
print("f1score")
print("%.6f" %f1)
cm = metrics.confusion_matrix(y_true, classes_x)
print("==============================================")
print (cm)

confusion matrix
----------------------------------------------
accuracy
0.996933
recall
0.996933
precision
0.996877
f1score
0.996888
[[17854     0     0     5     1     0]
 [    1   113     0     3     1     0]
 [    0     0     7     3     0     0]
 [    6     0     0  5581    12     3]
 [   17    12     0    10  2126     0]
 [    0     0     0     4     1     0]]
